## **Dependencies & Data Load**

In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = '{:.3f}'.format

In [3]:
# Data Loader

df = pd.read_excel('timezone.xlsx')

In [4]:
df.head()

,UserID,Name,Email,Timezone,Preferred_Working_Hours_Start,Preferred_Working_Hours_End,Avoid_Meeting_Before,Avoid_Meeting_After,Weekend_Availability,Public_Holidays_Exclusion,Feedback_Score
0,c6b98814-5fec-42cb-a20e-b85a2be7e164,Hannah Shaffer,jonesjohn@yahoo.com,UTC-6:00,9,19,6,20,No,Yes,Bad
1,21f15534-d71b-4a96-804c-f0e63b0f61ae,Jennifer Meyer,larryharrison@hotmail.com,UTC+1:00,6,17,9,21,Yes,No,Bad
2,c6d4d331-cb67-4ef3-840b-44701bd11166,Christopher Blankenship,bphelps@barber-smith.com,UTC-5:00,7,22,5,23,Yes,Yes,Bad
3,022a6d34-1837-48a2-8da9-1ab28b534e60,Tina Nelson,cadkins@lopez.com,UTC+1:00,6,16,5,22,Yes,Yes,Neutral
4,f9901851-cce4-49fb-a05b-2dbb0a352904,Dana Jones,higginsjames@hotmail.com,UTC+9:00,7,21,7,20,No,Yes,Neutral


In [14]:
for i, j in enumerate(df.columns):
    print(f'Column {i}  ==>  {j}')

Column 0  ==>  UserID
Column 1  ==>  Name
Column 2  ==>  Email
Column 3  ==>  Timezone
Column 4  ==>  Preferred_Working_Hours_Start
Column 5  ==>  Preferred_Working_Hours_End
Column 6  ==>  Avoid_Meeting_Before
Column 7  ==>  Avoid_Meeting_After
Column 8  ==>  Weekend_Availability
Column 9  ==>  Public_Holidays_Exclusion
Column 10  ==>  Feedback_Score


## **Scheduler**

In [16]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import pytz
from datetime import datetime, timedelta

class MeetingTimeScheduler:
    def __init__(self):
        # List of time zones to simulate global team
        self.time_zones = [
            'US/Pacific', 'US/Eastern', 'Europe/London', 
            'Asia/Tokyo', 'Australia/Sydney', 'Asia/Dubai'
        ]
        
        # Typical work hours (9 AM to 5 PM in local time)
        self.work_hours = (9, 17)
    
    def generate_synthetic_data(self, num_samples=1000):
        """
        Generate synthetic meeting schedule data
        
        Columns:
        - Time zones of participants
        - Workday start/end times
        - Meeting duration
        - Preferred meeting times
        - Team productivity score
        """
        data = []
        
        for _ in range(num_samples):
            # Randomly select number of participants (2-6)
            num_participants = random.randint(2, 6)
            
            # Randomly select time zones
            meeting_zones = random.sample(self.time_zones, num_participants)
            
            # Convert time zones to local times
            local_times = []
            for tz_name in meeting_zones:
                tz = pytz.timezone(tz_name)
                # Generate a random time within work hours
                local_hour = random.randint(self.work_hours[0], self.work_hours[1])
                local_time = datetime.now(tz).replace(
                    hour=local_hour, 
                    minute=random.randint(0, 59), 
                    second=0, 
                    microsecond=0
                )
                local_times.append(local_time)
            
            # Calculate time differences
            utc_times = [t.astimezone(pytz.UTC) for t in local_times]
            time_diffs = [(t - utc_times[0]).total_seconds() / 3600 for t in utc_times]
            
            # Meeting duration
            meeting_duration = random.randint(30, 120)
            
            # Productivity score (based on time alignment)
            time_spread = max(time_diffs) - min(time_diffs)
            productivity_score = max(0, 100 - (time_spread * 5))
            
            # Compile data point
            data.append({
                'num_participants': num_participants,
                'time_zones': ','.join(meeting_zones),
                'time_differences': ','.join(map(str, time_diffs)),
                'meeting_duration': meeting_duration,
                'productivity_score': productivity_score
            })
        
        return pd.DataFrame(data)
    
    def prepare_data(self, df):
        """
        Prepare data for machine learning
        """
        # One-hot encode time zones
        zone_dummies = df['time_zones'].str.get_dummies(sep=',')
        time_diff_features = df['time_differences'].str.split(',', expand=True).astype(float)
        time_diff_features.columns = [f'time_diff_{i}' for i in range(time_diff_features.shape[1])]
        
        # Combine features
        X = pd.concat([
            df[['num_participants', 'meeting_duration']],
            zone_dummies,
            time_diff_features
        ], axis=1)
        
        y = df['productivity_score']
        
        return train_test_split(X, y, test_size=0.2, random_state=42)
    
    def train_model(self, X_train, y_train):
        """
        Train Random Forest Regressor
        """
        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        
        # Train model
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train_scaled, y_train)
        
        return model, scaler
    
    def predict_best_meeting_time(self, model, scaler, new_meeting_info):
        """
        Predict the best meeting time based on team composition
        
        new_meeting_info should be a dictionary with:
        - time_zones: list of time zones
        - num_participants: number of participants
        - meeting_duration: expected meeting duration
        """
        # Prepare input data similar to training data
        zone_dummies = pd.get_dummies(new_meeting_info['time_zones'], prefix='zone')
        
        # Calculate time differences
        tz_objects = [pytz.timezone(tz) for tz in new_meeting_info['time_zones']]
        utc_times = [datetime.now(tz).replace(hour=12, minute=0, second=0, microsecond=0).astimezone(pytz.UTC) for tz in tz_objects]
        time_diffs = [(t - utc_times[0]).total_seconds() / 3600 for t in utc_times]
        time_diff_features = pd.DataFrame([time_diffs], columns=[f'time_diff_{i}' for i in range(len(time_diffs))])
        
        # Combine features
        input_data = pd.concat([
            pd.DataFrame({
                'num_participants': [new_meeting_info['num_participants']],
                'meeting_duration': [new_meeting_info['meeting_duration']]
            }),
            zone_dummies,
            time_diff_features
        ], axis=1)
        
        # Ensure all columns match training data
        input_data = input_data.reindex(columns=X_train.columns, fill_value=0)
        
        # Scale and predict
        input_scaled = scaler.transform(input_data)
        productivity_score = model.predict(input_scaled)[0]
        
        return productivity_score

In [17]:
# Main execution
scheduler = MeetingTimeScheduler()

# Generate synthetic data
df = scheduler.generate_synthetic_data()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   num_participants    1000 non-null   int64  
 1   time_zones          1000 non-null   object 
 2   time_differences    1000 non-null   object 
 3   meeting_duration    1000 non-null   int64  
 4   productivity_score  1000 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 39.2+ KB


In [18]:
df

,num_participants,time_zones,time_differences,meeting_duration,productivity_score
0,2,"Asia/Dubai,Asia/Tokyo","0.0,-7.733333333333333",105,61.333
1,6,"Australia/Sydney,US/Pacific,Europe/London,Asia...","0.0,15.6,14.233333333333333,10.1,-1.6666666666...",46,13.667
2,6,"Asia/Dubai,Asia/Tokyo,Australia/Sydney,US/East...","0.0,-12.4,-8.583333333333334,6.066666666666666...",82,7.667
3,3,"US/Pacific,Asia/Dubai,Asia/Tokyo","0.0,-20.416666666666668,-24.966666666666665",42,0.000
4,3,"US/Pacific,Australia/Sydney,Europe/London","0.0,-20.583333333333332,-10.783333333333333",45,0.000
...,...,...,...,...,...
995,6,"Australia/Sydney,Asia/Tokyo,US/Pacific,Asia/Du...","0.0,6.883333333333334,16.9,7.316666666666666,1...",37,3.417
996,5,"Asia/Dubai,Australia/Sydney,US/Eastern,Europe/...","0.0,-7.35,5.766666666666667,-0.116666666666666...",34,14.167
997,4,"Europe/London,Asia/Tokyo,US/Pacific,US/Eastern","0.0,-13.116666666666667,4.466666666666667,-0.1...",92,12.083
998,2,"Australia/Sydney,Europe/London","0.0,17.833333333333332",78,10.833


In [19]:
# Prepare data for machine learning
X_train, X_test, y_train, y_test = scheduler.prepare_data(df)

# Train the model
model, scaler = scheduler.train_model(X_train, y_train)

# Example of predicting best meeting time
new_meeting = {
    'time_zones': ['US/Pacific', 'Europe/London', 'Asia/Tokyo'],
    'num_participants': 3,
    'meeting_duration': 60
}

best_time_score = scheduler.predict_best_meeting_time(model, scaler, new_meeting)
print(f"Predicted Meeting Productivity Score: {best_time_score:.2f}")

Predicted Meeting Productivity Score: 23.79


## **Optimizer**

In [20]:
import pytz
from datetime import datetime, timedelta
from typing import List, Dict
import random

class MeetingTimeOptimizer:
    def __init__(self):
        # Predefined work hours (9 AM to 5 PM) for each time zone
        self.work_hours = {
            'US/Pacific': (9, 17),
            'US/Eastern': (9, 17),
            'Europe/London': (9, 17),
            'Asia/Tokyo': (9, 17),
            'Australia/Sydney': (9, 17),
            'Asia/Dubai': (9, 17),
            'Asia/Singapore': (9, 17)
        }
    
    def generate_employee_data(self, num_employees: int) -> pd.DataFrame:
        """
        Generate synthetic employee data with time zones
        """
        time_zones = list(self.work_hours.keys())
        
        employees = []
        for emp_id in range(1, num_employees + 1):
            # Randomly select a time zone for each employee
            timezone = random.choice(time_zones)
            
            employees.append({
                'employee_id': emp_id,
                'timezone': timezone
            })
        
        return pd.DataFrame(employees)
    
    def find_optimal_meeting_time(self, employee_data: pd.DataFrame) -> Dict[str, str]:
        """
        Find the optimal meeting time across different time zones
        
        Returns:
        - Optimal UTC time
        - Local times for each employee
        """
        # Convert all times to UTC for comparison
        current_utc = datetime.now(pytz.UTC)
        
        # Calculate possible meeting times
        optimal_times = []
        
        # Try times across the day in 1-hour increments
        for hour in range(24):
            test_utc_time = current_utc.replace(hour=hour, minute=0, second=0, microsecond=0)
            
            # Check if this time works for all employees
            time_compatibility = self._check_time_compatibility(test_utc_time, employee_data)
            
            if time_compatibility['is_compatible']:
                optimal_times.append({
                    'utc_time': test_utc_time,
                    'compatibility_score': time_compatibility['score']
                })
        
        # If no perfect time, find the best compromise
        if not optimal_times:
            optimal_times = self._find_best_compromise_time(employee_data)
        
        # Sort by compatibility score
        best_time = max(optimal_times, key=lambda x: x['compatibility_score'])
        
        # Convert to local times for each employee
        local_times = {}
        for _, employee in employee_data.iterrows():
            emp_tz = pytz.timezone(employee['timezone'])
            local_time = best_time['utc_time'].astimezone(emp_tz)
            local_times[employee['employee_id']] = local_time.strftime('%Y-%m-%d %H:%M %Z')
        
        return {
            'utc_time': best_time['utc_time'].strftime('%Y-%m-%d %H:%M UTC'),
            'local_times': local_times
        }
    
    def _check_time_compatibility(self, utc_time: datetime, employee_data: pd.DataFrame) -> Dict:
        """
        Check if a given UTC time is compatible with all employees' work hours
        """
        total_compatibility = 0
        is_compatible = True
        
        for _, employee in employee_data.iterrows():
            # Convert UTC time to employee's local timezone
            emp_tz = pytz.timezone(employee['timezone'])
            local_time = utc_time.astimezone(emp_tz)
            
            # Check if local time is within work hours
            work_start, work_end = self.work_hours[employee['timezone']]
            
            if work_start <= local_time.hour < work_end:
                # Time is within work hours
                total_compatibility += 1
            else:
                is_compatible = False
                # Penalize times outside work hours
                total_compatibility -= 1
        
        # Calculate compatibility score
        compatibility_score = total_compatibility / len(employee_data)
        
        return {
            'is_compatible': is_compatible,
            'score': compatibility_score
        }
    
    def _find_best_compromise_time(self, employee_data: pd.DataFrame) -> List[Dict]:
        """
        Find the best compromise time when no perfect time exists
        """
        compromise_times = []
        
        # Try different UTC hours
        for hour in range(24):
            current_utc = datetime.now(pytz.UTC).replace(hour=hour, minute=0, second=0, microsecond=0)
            
            # Evaluate time compatibility
            compatibility = self._check_time_compatibility(current_utc, employee_data)
            
            compromise_times.append({
                'utc_time': current_utc,
                'compatibility_score': compatibility['score']
            })
        
        return compromise_times

In [21]:
# Initialize the optimizer
optimizer = MeetingTimeOptimizer()

# Generate employee data (6 employees with different time zones)
employee_data = optimizer.generate_employee_data(6)
print("Employee Time Zones:")
print(employee_data)

Employee Time Zones:
   employee_id          timezone
0            1        Asia/Tokyo
1            2        US/Pacific
2            3  Australia/Sydney
3            4        US/Eastern
4            5    Asia/Singapore
5            6     Europe/London


In [22]:
# Find optimal meeting time
optimal_meeting = optimizer.find_optimal_meeting_time(employee_data)

print("Optimal Meeting Time:")
print(f"UTC Time: {optimal_meeting['utc_time']}")
print("\nLocal Times for Employees:")
for emp_id, local_time in optimal_meeting['local_times'].items():
    print(f"Employee {emp_id}: {local_time}")

Optimal Meeting Time:
UTC Time: 2024-11-30 00:00 UTC

Local Times for Employees:
Employee 1: 2024-11-30 09:00 JST
Employee 2: 2024-11-29 16:00 PST
Employee 3: 2024-11-30 11:00 AEDT
Employee 4: 2024-11-29 19:00 EST
Employee 5: 2024-11-30 08:00 +08
Employee 6: 2024-11-30 00:00 GMT


## **KMeans Algorithm**

In [25]:
from typing import List, Dict, Optional
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

class AdvancedMeetingScheduler:
    def __init__(self, employee_database_path: str):
        """
        Initialize scheduler with full employee database
        
        Args:
            employee_database_path: Path to CSV with employee details
        """
        # Load full employee database
        self.employee_database = self._load_employee_database(employee_database_path)
        
        # Work hour preferences (configurable)
        self.work_hours = {
            'US/Pacific': (8, 17),
            'US/Eastern': (9, 18),
            'Europe/London': (9, 17),
            'Asia/Tokyo': (9, 18),
            'Australia/Sydney': (8, 17),
            'Asia/Dubai': (8, 17),
            'Asia/Singapore': (9, 18)
        }
        
        # Additional employee attributes for advanced scheduling
        self.employee_attributes = [
            'communication_preference',
            'team',
            'seniority_level',
            'meeting_frequency'
        ]
    
    def _load_employee_database(self, path: str) -> pd.DataFrame:
        """
        Load and preprocess employee database
        
        Returns:
            DataFrame with employee details
        """
        df = pd.read_csv(path)
        
        # Ensure required columns exist
        required_columns = ['employee_id', 'timezone', 'department']
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"Missing required column: {col}")
        
        return df
    
    def select_meeting_participants(
        self, 
        num_participants: int, 
        criteria: Optional[Dict] = None
    ) -> pd.DataFrame:
        """
        Select participants based on specific criteria
        
        Args:
            num_participants: Number of people for the meeting
            criteria: Optional filtering criteria
        
        Returns:
            DataFrame of selected participants
        """
        filtered_df = self.employee_database.copy()
        
        # Apply filtering criteria if provided
        if criteria:
            for key, value in criteria.items():
                if key in filtered_df.columns:
                    filtered_df = filtered_df[filtered_df[key] == value]
        
        # Randomly select participants if more than required
        if len(filtered_df) > num_participants:
            selected_participants = filtered_df.sample(n=num_participants)
        else:
            selected_participants = filtered_df
        
        return selected_participants
    
    def advanced_time_optimization(
        self, 
        participants: pd.DataFrame, 
        meeting_duration: int = 60,
        preferred_time_ranges: Optional[List[Tuple[int, int]]] = None
    ) -> Dict:
        """
        Advanced meeting time optimization
        
        Args:
            participants: DataFrame of selected participants
            meeting_duration: Meeting length in minutes
            preferred_time_ranges: Optional list of preferred time ranges
        
        Returns:
            Optimal meeting time details
        """
        # Calculate time compatibility matrix
        compatibility_matrix = self._create_compatibility_matrix(
            participants, 
            meeting_duration, 
            preferred_time_ranges
        )
        
        # Find optimal time slot
        optimal_slot = self._find_optimal_time_slot(
            compatibility_matrix, 
            participants
        )
        
        # Convert to local times
        local_times = self._convert_to_local_times(
            optimal_slot['utc_time'], 
            participants
        )
        
        return {
            'utc_time': optimal_slot['utc_time'].strftime('%Y-%m-%d %H:%M UTC'),
            'local_times': local_times,
            'compatibility_score': optimal_slot['compatibility_score']
        }
    
    def _create_compatibility_matrix(
        self, 
        participants: pd.DataFrame, 
        meeting_duration: int,
        preferred_time_ranges: Optional[List[Tuple[int, int]]] = None
    ) -> Dict:
        """
        Create a compatibility matrix for potential meeting times
        
        Args:
            participants: Selected participants
            meeting_duration: Meeting length in minutes
            preferred_time_ranges: Optional preferred time ranges
        
        Returns:
            Compatibility matrix of time slots
        """
        compatibility_matrix = {}
        
        # Define time ranges to check
        time_ranges = preferred_time_ranges or [(0, 24)]
        
        # Check compatibility for each time range
        for start_hour, end_hour in time_ranges:
            for hour in range(start_hour, end_hour):
                utc_time = datetime.now(pytz.UTC).replace(
                    hour=hour, minute=0, second=0, microsecond=0
                )
                
                compatibility = self._evaluate_time_compatibility(
                    utc_time, 
                    participants, 
                    meeting_duration
                )
                
                compatibility_matrix[utc_time] = compatibility
        
        return compatibility_matrix
    
    def _evaluate_time_compatibility(
        self, 
        utc_time: datetime, 
        participants: pd.DataFrame, 
        meeting_duration: int
    ) -> Dict:
        """
        Evaluate compatibility of a specific time slot
        
        Args:
            utc_time: Proposed UTC meeting time
            participants: Selected participants
            meeting_duration: Meeting length in minutes
        
        Returns:
            Compatibility score and details
        """
        total_compatibility = 0
        participant_details = {}
        
        for _, participant in participants.iterrows():
            # Convert to participant's local timezone
            tz = pytz.timezone(participant['timezone'])
            local_time = utc_time.astimezone(tz)
            
            # Check work hour compatibility
            work_start, work_end = self.work_hours.get(
                participant['timezone'], 
                (9, 17)  # Default if not specified
            )
            
            # Check if meeting falls within work hours
            is_compatible = (
                work_start <= local_time.hour < work_end - (meeting_duration // 60)
            )
            
            compatibility_score = 1 if is_compatible else 0
            total_compatibility += compatibility_score
            
            participant_details[participant['employee_id']] = {
                'local_time': local_time,
                'is_compatible': is_compatible
            }
        
        # Calculate overall compatibility
        overall_compatibility = total_compatibility / len(participants)
        
        return {
            'score': overall_compatibility,
            'participant_details': participant_details
        }
    
    def _find_optimal_time_slot(
        self, 
        compatibility_matrix: Dict, 
        participants: pd.DataFrame
    ) -> Dict:
        """
        Find the most optimal time slot
        
        Args:
            compatibility_matrix: Matrix of time slot compatibilities
            participants: Selected participants
        
        Returns:
            Most optimal time slot
        """
        # Sort compatibility matrix by score
        sorted_slots = sorted(
            compatibility_matrix.items(), 
            key=lambda x: x[1]['score'], 
            reverse=True
        )
        
        # Return the best time slot
        best_slot = sorted_slots[0]
        
        return {
            'utc_time': best_slot[0],
            'compatibility_score': best_slot[1]['score']
        }
    
    def _convert_to_local_times(
        self, 
        utc_time: datetime, 
        participants: pd.DataFrame
    ) -> Dict:
        """
        Convert UTC time to local times for participants
        
        Args:
            utc_time: Proposed UTC meeting time
            participants: Selected participants
        
        Returns:
            Dictionary of local times
        """
        local_times = {}
        
        for _, participant in participants.iterrows():
            tz = pytz.timezone(participant['timezone'])
            local_time = utc_time.astimezone(tz)
            
            local_times[participant['employee_id']] = local_time.strftime('%Y-%m-%d %H:%M %Z')
        
        return local_times

In [28]:
# Interactive Interface
def run_meeting_scheduler():
    # Path to employee database (you would replace this with your actual path)
    DATABASE_PATH = 'employee_database.csv'
    
    # Create scheduler instance
    scheduler = AdvancedMeetingScheduler(DATABASE_PATH)
    
    # Interactive input
    print("Advanced Meeting Scheduler")
    print("------------------------")
    
    # Get meeting parameters
    num_participants = int(input("How many participants do you need? (max 8): "))
    meeting_duration = int(input("Meeting duration in minutes: "))
    
    # Optional filtering criteria
    print("\nOptional Filtering Criteria (press enter to skip)")
    filtering_criteria = {}
    
    department = input("Filter by department (optional): ")
    if department:
        filtering_criteria['department'] = department
    
    # Select participants
    participants = scheduler.select_meeting_participants(
        num_participants, 
        criteria=filtering_criteria
    )
    
    # Optional time range preference
    time_range_input = input("Preferred time range? (e.g., 9-17 or leave blank): ")
    preferred_time_ranges = None
    if time_range_input:
        start, end = map(int, time_range_input.split('-'))
        preferred_time_ranges = [(start, end)]
    
    # Find optimal meeting time
    optimal_meeting = scheduler.advanced_time_optimization(
        participants, 
        meeting_duration,
        preferred_time_ranges
    )
    
    # Display results
    print("\n--- Optimal Meeting Time ---")
    print(f"UTC Time: {optimal_meeting['utc_time']}")
    print(f"Compatibility Score: {optimal_meeting['compatibility_score']:.2f}")
    
    print("\nLocal Times for Participants:")
    for emp_id, local_time in optimal_meeting['local_times'].items():
        print(f"Employee {emp_id}: {local_time}")

In [29]:
# Run
run_meeting_scheduler()

Advanced Meeting Scheduler
------------------------


How many participants do you need? (max 8):  6
Meeting duration in minutes:  60



Optional Filtering Criteria (press enter to skip)


Filter by department (optional):  
Preferred time range? (e.g., 9-17 or leave blank):  



--- Optimal Meeting Time ---
UTC Time: 2024-11-30 04:00 UTC
Compatibility Score: 0.83

Local Times for Participants:
Employee 1: 2024-11-29 20:00 PST
Employee 7: 2024-11-30 12:00 +08
Employee 12: 2024-11-30 13:00 JST
Employee 5: 2024-11-30 08:00 +04
Employee 18: 2024-11-30 15:00 AEDT
Employee 17: 2024-11-30 12:00 +08


## **KMeans Implementation 2**

In [30]:
class AdvancedMeetingScheduler:
    def __init__(self, employee_database_path: str):
        """
        Initialize scheduler with full employee database
        
        Args:
            employee_database_path: Path to CSV with employee details
        """
        # Load full employee database
        self.employee_database = self._load_employee_database(employee_database_path)
        
        # Work hour preferences (configurable)
        self.work_hours = {
            'US/Pacific': (8, 17),
            'US/Eastern': (9, 18),
            'Europe/London': (9, 17),
            'Asia/Tokyo': (9, 18),
            'Australia/Sydney': (8, 17),
            'Asia/Dubai': (8, 17),
            'Asia/Singapore': (9, 18)
        }
    
    def _load_employee_database(self, path: str) -> pd.DataFrame:
        """
        Load and preprocess employee database
        
        Returns:
            DataFrame with employee details
        """
        df = pd.read_csv(path)
        
        # Ensure required columns exist
        required_columns = ['employee_id', 'timezone', 'department']
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"Missing required column: {col}")
        
        return df
    
    def select_participants_by_ids(self, employee_ids: List[int]) -> pd.DataFrame:
        """
        Select participants based on specific employee IDs
        
        Args:
            employee_ids: List of employee IDs to include in the meeting
        
        Returns:
            DataFrame of selected participants
        """
        # Filter employees by provided IDs
        selected_participants = self.employee_database[
            self.employee_database['employee_id'].isin(employee_ids)
        ]
        
        # Validate that all requested IDs exist
        missing_ids = set(employee_ids) - set(selected_participants['employee_id'])
        if missing_ids:
            raise ValueError(f"Employee IDs not found: {missing_ids}")
        
        return selected_participants
    
    def find_optimal_meeting_time(
        self, 
        participants: pd.DataFrame, 
        meeting_duration: int = 60,
        preferred_time_ranges: Optional[List[tuple]] = None
    ) -> Dict:
        """
        Find optimal meeting time for selected participants
        
        Args:
            participants: DataFrame of selected participants
            meeting_duration: Meeting length in minutes
            preferred_time_ranges: Optional list of preferred time ranges
        
        Returns:
            Optimal meeting time details
        """
        # Display selected participants
        print("Selected Participants:")
        for _, participant in participants.iterrows():
            print(f"ID: {participant['employee_id']} - {participant['first_name']} {participant['last_name']} ({participant['timezone']})")
        
        # Create compatibility matrix
        compatibility_matrix = self._create_compatibility_matrix(
            participants, 
            meeting_duration, 
            preferred_time_ranges
        )
        
        # Find optimal time slot
        optimal_slot = self._find_optimal_time_slot(
            compatibility_matrix, 
            participants
        )
        
        # Convert to local times
        local_times = self._convert_to_local_times(
            optimal_slot['utc_time'], 
            participants
        )
        
        return {
            'utc_time': optimal_slot['utc_time'].strftime('%Y-%m-%d %H:%M UTC'),
            'local_times': local_times,
            'compatibility_score': optimal_slot['compatibility_score']
        }
    
    def _create_compatibility_matrix(
        self, 
        participants: pd.DataFrame, 
        meeting_duration: int,
        preferred_time_ranges: Optional[List[tuple]] = None
    ) -> Dict:
        """
        Create a compatibility matrix for potential meeting times
        """
        compatibility_matrix = {}
        
        # Define time ranges to check
        time_ranges = preferred_time_ranges or [(0, 24)]
        
        # Check compatibility for each time range
        for start_hour, end_hour in time_ranges:
            for hour in range(start_hour, end_hour):
                utc_time = datetime.now(pytz.UTC).replace(
                    hour=hour, minute=0, second=0, microsecond=0
                )
                
                compatibility = self._evaluate_time_compatibility(
                    utc_time, 
                    participants, 
                    meeting_duration
                )
                
                compatibility_matrix[utc_time] = compatibility
        
        return compatibility_matrix
    
    def _evaluate_time_compatibility(
        self, 
        utc_time: datetime, 
        participants: pd.DataFrame, 
        meeting_duration: int
    ) -> Dict:
        """
        Evaluate compatibility of a specific time slot
        """
        total_compatibility = 0
        participant_details = {}
        
        for _, participant in participants.iterrows():
            # Convert to participant's local timezone
            tz = pytz.timezone(participant['timezone'])
            local_time = utc_time.astimezone(tz)
            
            # Check work hour compatibility
            work_start, work_end = self.work_hours.get(
                participant['timezone'], 
                (9, 17)  # Default if not specified
            )
            
            # Check if meeting falls within work hours
            is_compatible = (
                work_start <= local_time.hour < work_end - (meeting_duration // 60)
            )
            
            compatibility_score = 1 if is_compatible else 0
            total_compatibility += compatibility_score
            
            participant_details[participant['employee_id']] = {
                'local_time': local_time,
                'is_compatible': is_compatible
            }
        
        # Calculate overall compatibility
        overall_compatibility = total_compatibility / len(participants)
        
        return {
            'score': overall_compatibility,
            'participant_details': participant_details
        }
    
    def _find_optimal_time_slot(
        self, 
        compatibility_matrix: Dict, 
        participants: pd.DataFrame
    ) -> Dict:
        """
        Find the most optimal time slot
        """
        # Sort compatibility matrix by score
        sorted_slots = sorted(
            compatibility_matrix.items(), 
            key=lambda x: x[1]['score'], 
            reverse=True
        )
        
        # Return the best time slot
        best_slot = sorted_slots[0]
        
        return {
            'utc_time': best_slot[0],
            'compatibility_score': best_slot[1]['score']
        }
    
    def _convert_to_local_times(
        self, 
        utc_time: datetime, 
        participants: pd.DataFrame
    ) -> Dict:
        """
        Convert UTC time to local times for participants
        """
        local_times = {}
        
        for _, participant in participants.iterrows():
            tz = pytz.timezone(participant['timezone'])
            local_time = utc_time.astimezone(tz)
            
            local_times[participant['employee_id']] = local_time.strftime('%Y-%m-%d %H:%M %Z')
        
        return local_times

In [31]:
# Main function to run the meeting scheduler
def run_meeting_scheduler():
    # Path to employee database
    DATABASE_PATH = 'employee_database.csv'
    
    # Create scheduler instance
    scheduler = AdvancedMeetingScheduler(DATABASE_PATH)
    
    # Interactive input for employee IDs
    print("Advanced Meeting Scheduler")
    print("------------------------")
    
    # Get employee IDs
    employee_ids_input = input("Enter employee IDs for the meeting (comma-separated, e.g., 1,3,4,5,8): ")
    
    # Convert input to list of integers
    try:
        employee_ids = [int(id.strip()) for id in employee_ids_input.split(',')]
    except ValueError:
        print("Invalid input. Please enter valid employee IDs.")
        return
    
    # Get meeting duration
    meeting_duration = int(input("Meeting duration in minutes: "))
    
    # Optional time range preference
    time_range_input = input("Preferred time range? (e.g., 9-17 or leave blank): ")
    preferred_time_ranges = None
    if time_range_input:
        start, end = map(int, time_range_input.split('-'))
        preferred_time_ranges = [(start, end)]
    
    try:
        # Select participants by IDs
        participants = scheduler.select_participants_by_ids(employee_ids)
        
        # Find optimal meeting time
        optimal_meeting = scheduler.find_optimal_meeting_time(
            participants, 
            meeting_duration,
            preferred_time_ranges
        )
        
        # Display results
        print("\n--- Optimal Meeting Time ---")
        print(f"UTC Time: {optimal_meeting['utc_time']}")
        print(f"Compatibility Score: {optimal_meeting['compatibility_score']:.2f}")
        
        print("\nLocal Times for Participants:")
        for emp_id, local_time in optimal_meeting['local_times'].items():
            print(f"Employee {emp_id}: {local_time}")
    
    except ValueError as e:
        print(f"Error: {e}")

In [33]:
# Run
run_meeting_scheduler()

Advanced Meeting Scheduler
------------------------


Enter employee IDs for the meeting (comma-separated, e.g., 1,3,4,5,8):  2,3,5,7
Meeting duration in minutes:  60
Preferred time range? (e.g., 9-17 or leave blank):  8-15


Selected Participants:
ID: 2 - Emily Johnson (US/Eastern)
ID: 3 - Hiroshi Tanaka (Asia/Tokyo)
ID: 5 - Mohammed Al-Fayed (Asia/Dubai)
ID: 7 - Chen Wei (Asia/Singapore)

--- Optimal Meeting Time ---
UTC Time: 2024-11-30 08:00 UTC
Compatibility Score: 0.50

Local Times for Participants:
Employee 2: 2024-11-30 03:00 EST
Employee 3: 2024-11-30 17:00 JST
Employee 5: 2024-11-30 12:00 +04
Employee 7: 2024-11-30 16:00 +08


## **Implementation 2**

In [64]:
import pandas as pd
import numpy as np
from typing import List, Dict
from datetime import datetime, timedelta
import pytz

class MeetingTimeOptimizer:
    def __init__(self, data: pd.DataFrame):
        """
        Initialize the Meeting Time Optimizer
        
        Args:
            data (pd.DataFrame): DataFrame containing employee information
        """
        self.df = data
        self.timezones = {}
        self._parse_timezones()
    
    def _parse_timezones(self):
        """
        Parse and standardize timezone information
        """
        for _, row in self.df.iterrows():
            try:
                # Convert timezone strings to pytz objects
                tz = pytz.timezone(row['timezone'])
                self.timezones[row['employee_id']] = {
                    'timezone': tz,
                    'utc_offset': self._get_utc_offset(tz),
                    'location': row['location'],
                    'department': row['department'],
                    'seniority_level': row['seniority_level']
                }
            except Exception as e:
                print(f"Error processing timezone for {row['employee_id']}: {e}")
    
    def _get_utc_offset(self, tz) -> float:
        """
        Calculate UTC offset for a given timezone
        
        Args:
            tz (pytz.timezone): Timezone object
        
        Returns:
            float: UTC offset in hours
        """
        # Get current UTC offset
        return datetime.now(tz).utcoffset().total_seconds() / 3600
    
    def recommend_meeting_time(self, employee_ids: List[int]) -> Dict:
        """
        Recommend optimal meeting time considering multiple constraints
        
        Args:
            employee_ids (List[int]): List of employee IDs to include in meeting
        
        Returns:
            Dict: Recommended meeting details
        """
        # Validate employee IDs
        valid_employees = [emp_id for emp_id in employee_ids 
                           if emp_id in self.timezones]
        
        if not valid_employees:
            raise ValueError("No valid employees found")
        
        # Extract timezone information for selected employees
        selected_timezones = {
            emp_id: self.timezones[emp_id] for emp_id in valid_employees
        }
        
        # Calculate offset statistics
        offsets = [info['utc_offset'] for info in selected_timezones.values()]
        
        # Optimal time calculation strategies
        def calculate_optimal_time():
            # Strategy 1: Median offset
            median_offset = np.median(offsets)
            
            # Strategy 2: Weighted offset considering seniority and department
            def weight_offset(emp_id):
                emp_info = selected_timezones[emp_id]
                seniority_weights = {
                    'junior': 0.7,
                    'mid': 1.0,
                    'senior': 1.3
                }
                return (
                    emp_info['utc_offset'] * 
                    seniority_weights.get(emp_info['seniority_level'], 1.0)
                )
            
            weighted_offsets = [weight_offset(emp_id) for emp_id in valid_employees]
            weighted_median = np.median(weighted_offsets)
            
            return weighted_median
        
        # Determine optimal meeting time
        optimal_offset = calculate_optimal_time()
        
        # Validate time is within reasonable working hours
        def validate_working_hours(offset):
            # Assume standard working hours are 9 AM - 5 PM
            return -6 <= offset <= 12
        
        if not validate_working_hours(optimal_offset):
            # Fallback to a more moderate time
            optimal_offset = max(-6, min(optimal_offset, 12))
        
        # Prepare recommendation details
        recommendation = {
            'recommended_utc_offset': f'UTC{int(optimal_offset):+d}',
            'participants': [],
            'offset_calculation': {
                'median_offset': np.median(offsets),
                'weighted_median_offset': optimal_offset
            }
        }
        
        # Add participant details
        for emp_id in valid_employees:
            emp_info = selected_timezones[emp_id]
            recommendation['participants'].append({
                'employee_id': emp_id,
                'timezone': str(emp_info['timezone']),
                'location': emp_info['location'],
                'local_time_at_meeting': self._convert_to_local_time(
                    optimal_offset, 
                    emp_info['timezone']
                )
            })
        
        return recommendation
    
    def _convert_to_local_time(self, target_offset, local_tz):
        """
        Convert target UTC time to local timezone
        
        Args:
            target_offset (float): Target UTC offset
            local_tz (pytz.timezone): Local timezone
        
        Returns:
            str: Formatted local time
        """
        # Create a UTC time with the target offset
        utc_time = datetime.utcnow().replace(
            hour=10, minute=0, second=0, microsecond=0  # Assume 10 AM UTC
        )
        utc_time += timedelta(hours=target_offset)
        
        # Convert to local timezone
        local_time = utc_time.astimezone(local_tz)
        
        return local_time.strftime('%I:%M %p')

In [87]:
def scheduler(emp_list:list):
    # Load data (replace with your actual data loading method)
    data = pd.read_excel('globalsync.xlsx')  # Or use the provided dataframe
    
    # Create optimizer
    optimizer = MeetingTimeOptimizer(data)
    
    # Example usage
    try:
        # Recommend meeting time for specific employees
        recommendation = optimizer.recommend_meeting_time(emp_list)
        
        # Print recommendation details
        print("========== Meeting Time Recommendation ==========")
        print(f"Recommended UTC Offset: {recommendation['recommended_utc_offset']} \n")
        print("========== Participant Local Times ==========")
        for participant in recommendation['participants']:
            print(f"Employee {participant['employee_id']} ({participant['timezone']}):")
            print(f"  Location: {participant['location']}")
            print(f"  Local Meeting Time: {participant['local_time_at_meeting']}\n")
        
        print("Offset Calculation Details:")
        print(recommendation['offset_calculation'])
    
    except Exception as e:
        print(f"Error in meeting time recommendation: {e}")

In [88]:
# Run
scheduler([21, 28])

========== Meeting Time Recommendation ==========
Recommended UTC Offset: UTC+1 

========== Participant Local Times ==========
Employee 21 (Europe/Berlin):
  Location: Hamburg, Germany
  Local Meeting Time: 06:30 AM

Employee 28 (Europe/Prague):
  Location: Prague, Czech Republic
  Local Meeting Time: 06:30 AM

Offset Calculation Details:
{'median_offset': 1.0, 'weighted_median_offset': 1.0}
